# Naive demonstration of a Markov Language Model 

We build a model that will compute
$$
P(w_1, \dots, w_n) = \prod\limits_{i=1}^{n} P(w_i \mid w_{i-m}, \dots w_{i-1})
$$

In [1]:
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
import numpy as np 
import pandas as pd 

In [5]:
idx = {
    'i': {'am': 100, 'have': 50, 'will': 10},
    'am': {'happy': 2, 'sad': 5},

}
# P(am | I)
print(idx['i']['am'] / sum(idx['i'].values()))
print(idx['i']['have'] / sum(idx['i'].values()))
print(idx['i']['will'] / sum(idx['i'].values()))

0.625
0.3125
0.0625


### Gather some data

In [6]:
from sklearn.datasets import fetch_20newsgroups

In [7]:
download_dir = '/Users/flint/Data/sklearn/'
subset = 'all'
remove = ('headers', 'footers', 'quotes')
data = fetch_20newsgroups(data_home=download_dir, subset=subset, remove=remove)

In [8]:
print(data.data[0][:250], '...')
print('--------')
print(data.target[0], data.target_names[data.target[0]])



I am sure some bashers of Pens fans are pretty confused about the lack
of any kind of posts about the recent Pens massacre of the Devils. Actually,
I am  bit puzzled too and a bit relieved. However, I am going to put an end
to non-PIttsburghers' re ...
--------
10 rec.sport.hockey


### Training ML

In [9]:
import sys 
sys.path.append('../nlp/')
from nlp.markovlm import NaiveMarkovLM

In [10]:
lm = NaiveMarkovLM(n=3)

In [11]:
lm.train(data.data[:10])

In [12]:
lm.index.T.head()

i  actually  however  man  jagr   he  bowman  pens    !   my  \
#S   #S    9.0       1.0      1.0  1.0   1.0  1.0     1.0   1.0  1.0  2.0   
     i     1.0       0.0      0.0  0.0   0.0  0.0     0.0   0.0  0.0  0.0   
i    am    0.0       0.0      0.0  0.0   0.0  0.0     0.0   0.0  0.0  0.0   
am   sure  0.0       0.0      0.0  0.0   0.0  0.0     0.0   0.0  0.0  0.0   
sure some  0.0       0.0      0.0  0.0   0.0  0.0     0.0   0.0  0.0  0.0   

           ...  made  3-4  flaming  wings  pizza  hut  commercial  tlu/a  gic  \
#S   #S    ...   0.0  0.0      0.0    0.0    0.0  0.0         0.0    0.0  0.0   
     i     ...   0.0  0.0      0.0    0.0    0.0  0.0         0.0    0.0  0.0   
i    am    ...   0.0  0.0      0.0    0.0    0.0  0.0         0.0    0.0  0.0   
am   sure  ...   0.0  0.0      0.0    0.0    0.0  0.0         0.0    0.0  0.0   
sure some  ...   0.0  0.0      0.0    0.0    0.0  0.0         0.0    0.0  0.0   

           bait  
#S   #S     0.0  
     i      0.0  
i    am     0.0  
am   sure   0.0  
sure some   0.0  

[5 rows x 516 columns]

#### Compute conditional probabilities

In [13]:
seq = ('i', 'am', 'alfio')
print(lm.P(*seq, log=True))
seq = ('i', 'am', 'sure')
print(lm.P(*seq, log=True))

-1.791759469228055
-1.252762968495368


**Text probability**

In [14]:
text = sent_tokenize(data.data[0])[0].replace('\n', '')
print(text)
print(lm.joint_log_probability(text))

I am sure some bashers of Pens fans are pretty confused about the lackof any kind of posts about the recent Pens massacre of the Devils.
-4.882801922586371


#### Generate text

In [15]:
prefix = ('#S', '#S')
text = list(lm.generate(prefix=prefix, max_len=20))
print(" ".join(text))

#S #S that was new .... #E


## Applications

### Classification
1. we train a general model over the whole corpus
2. then, we clone the model into a specific model for each class 
3. we fine-tune the class-specific model for its class
4. given a text, we compute the text probability for each class-model in order to select the best 

In [16]:
global_lm = NaiveMarkovLM(n=3)
global_lm.train(documents=data.data)

In [17]:
print(list(global_lm.generate(max_len=20)))

['#S', '#S', 'the', 'azeri', 'town', 'on', 'a', 'level', 'consistent', 'with', 'that', '--', 'again', ',', 'you', 'can', 'obtain', 'the', 'above', 'represents', 'the']


**Cloning and fine tuning**

In [18]:
from tqdm.notebook import tqdm

In [19]:
class_models = {}
# Test with three classes only to speed up the process
run = list(enumerate(data.target_names[:3]))
for i, label in tqdm(run):
    class_docs = [data.data[j] for j, k in enumerate(data.target) if k==i]
    class_models[label] = global_lm.clone()
    # fine turning
    class_models[label].train(class_docs)

  0%|          | 0/3 [00:00<?, ?it/s]

In [20]:
print(class_models.keys())

dict_keys(['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc'])


**Classification**

In [21]:
count = 0
for i, doc in enumerate(data.data):
    label = data.target_names[data.target[i]]
    if label in class_models.keys():
        print("Correct class: ", label)
        print("Prediction")
        for pred, clm in class_models.items():
            log_p = clm.joint_log_probability(doc)
            print(pred, log_p)
        count += 1
        print("===========")
    if count > 10:
        break 

Correct class:  alt.atheism
Prediction
alt.atheism -121.81594594156735
comp.graphics -122.4464521136665
comp.os.ms-windows.misc -122.7542930595089
Correct class:  comp.graphics
Prediction
alt.atheism -928.400391920057
comp.graphics -935.5237164981246
comp.os.ms-windows.misc -946.5383220033161
Correct class:  comp.graphics
Prediction
alt.atheism -188.29301369058854
comp.graphics -189.01899862453735
comp.os.ms-windows.misc -192.11391284565815
Correct class:  alt.atheism
Prediction
alt.atheism -2314.665188087034
comp.graphics -2328.138985275299
comp.os.ms-windows.misc -2346.8211299736836
Correct class:  comp.graphics
Prediction
alt.atheism -434.15892132454286
comp.graphics -435.72816410439407
comp.os.ms-windows.misc -441.47625707730936
Correct class:  comp.os.ms-windows.misc
Prediction
alt.atheism -664.4213946267886
comp.graphics -667.7872258557901
comp.os.ms-windows.misc -674.5441184644216
Correct class:  comp.os.ms-windows.misc
Prediction
alt.atheism -330.9984957632746
comp.graphics -33

### Generation
1. we fine tune class-specific models as for classification
2. we generate texts to see if they are consistent with the class

In [22]:
prefix = ('i', 'am')
for label, clm in class_models.items():
    print(label)
    for i in range(4):
        print('\t', " ".join(list(clm.generate(prefix=prefix, max_len=20))))

alt.atheism
	 i am going , because as soon as possible if it was aggressively recruiting out of this british brigade was followed
	 i am using exactly the point of fire ( or gaps ) between ingestion and its cause is kept separate from
	 i am going to get parts . #E
	 i am almost all of the 15 february 1989 remnant merits careful study of the ` auto ' tranny in f1
comp.graphics
	 i am a novice to the clipper header/signature . #E
	 i am pretty sure theres an easy 4000 miles since new serialized , and it took almost two years behind the
	 i am hitting the caps-lock and spreading . #E
	 i am a few more sources relied upon the studies on this list answers many frequently asked questions on 77 occasions
comp.os.ms-windows.misc
	 i am interested to find some people consider the following : `` numerical recipes in c becaues of memory free ,
	 i am looking for specific cancer patient populations , as though the 805 can address to 'moody monthly ' ? 2p
	 i am not so with admirable verbal skill 